# ReesSaver Discriminator Agent

## Notes:
- Roughly 90% of our boards are unique
- Every time you call generate_data it gets new games/games in a different order?

In [1]:
import chess

import chess.svg
import cv2
from IPython.display import display, SVG

import numpy as np
import random
from tqdm import tqdm
from importlib import reload
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import utils
reload(utils)
import utils


from sklearn.model_selection import train_test_split

C:\Users\osama\anaconda3\lib\site-packages\torch\__init__.py:690: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [2]:
if torch.cuda.is_available():
    # Set default tensor type to CUDA tensors
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    
else:
    
    torch.set_default_tensor_type(torch.FloatTensor)
 
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

made_loader = False

True


In [3]:
def clear_cuda():
    
    input()
    
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            if obj.is_cuda:
                print(type(obj), obj.size(), obj.device)
                del obj
    torch.cuda.empty_cache()
    gc.collect()

In [4]:
clear_cuda()

<class 'torch.Tensor'> torch.Size([64]) cuda:0


C:\Users\osama\anaconda3\lib\site-packages\torch\distributed\distributed_c10d.py:359: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [32]:
from utils.Datasets import *

import utils.Dataloading
reload(utils.Dataloading)
from utils.Dataloading import *
from utils.Game_playing import *

import utils.Playing_agents
reload(utils.Playing_agents)
from utils.Playing_agents import *

from utils.CSV_data import *
from utils.Puzzles import *

In [34]:
class MLPv2_1(nn.Module):

    def __init__(self):

        super().__init__()
        self.conv1 = nn.Conv2d(14, 64, 3, 1, padding=1, padding_mode = 'zeros')

        self.layers = nn.ModuleList()

        self.depth = 6

        for _ in range(self.depth):
            self.layers.append(nn.Conv2d(64,64, 3, 1, padding=1, padding_mode = 'zeros'))
            self.layers.append(nn.BatchNorm2d(64))
            self.layers.append(nn.Conv2d(64,64, 3, 1, padding=1, padding_mode = 'zeros'))
            self.layers.append(nn.BatchNorm2d(64))

        self.linear = nn.Linear(4096, 128)

    def forward(self, x):

        x = self.conv1(x)

        for i in range(self.depth):
            j = i*4
            ph = x.clone()
            ph = self.layers[j](ph)
            ph = self.layers[j+1](ph)
            ph = F.relu(ph)
            ph = self.layers[j+2](ph)
            ph = self.layers[j+3](ph)

            x = x + ph
            x = F.relu(x)


        x = torch.flatten(x, start_dim=1)

        x = self.linear(x)

        minn, ila = x[:,:64], x[:,64:]

        return minn, ila

In [35]:
RDv2 = torch.load("Models/RDv2.3 CB.pt", map_location= device)

In [6]:
boards, meta, elo, moves, _, _, fens = generate_data("./Data/GAN_human_data.pgn", N = 40_000)
elo = [int(x) for x in elo]

0it [00:00, ?it/s]
100%|██████████| 40000/40000 [18:08<00:00, 36.74it/s]  


In [7]:
class generator_1(nn.Module):

    def __init__(self, conv_depth, hidden_size, hidden_depth):

        super().__init__()
        self.conv1 = nn.Conv2d(14, 64, 3, 1, padding=1, padding_mode = 'zeros')
        
        self.conv_layers = nn.ModuleList()
        self.hidden_layers = nn.ModuleList()
        
        self.conv_depth = conv_depth
        self.hidden_depth = hidden_depth
        self.hidden_size = hidden_size
        
        for _ in range(self.conv_depth):
            self.conv_layers.append(nn.Conv2d(64,64, 3, 1, padding=1, padding_mode = 'zeros'))
            self.conv_layers.append(nn.BatchNorm2d(64))
            self.conv_layers.append(nn.Conv2d(64,64, 3, 1, padding=1, padding_mode = 'zeros'))
            self.conv_layers.append(nn.BatchNorm2d(64))

        self.hidden_layers.append(nn.Linear(4096, hidden_size))
        
        for _ in range(self.hidden_depth - 2):

            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))

        self.hidden_layers.append(nn.Linear(hidden_size, 128))

    def forward(self, x):

        x = self.conv1(x)
        
        for i in range(self.conv_depth):
            j = i*4
            ph = x.clone()
            ph = self.conv_layers[j](ph)
            ph = self.conv_layers[j+1](ph)
            ph = F.relu(ph)
            ph = self.conv_layers[j+2](ph)
            ph = self.conv_layers[j+3](ph)
            
            x = x + ph
            x = F.relu(x)   
        
        x = torch.flatten(x, start_dim=1)
        
        for i in range(self.hidden_depth - 1):
            
            x = self.hidden_layers[i](x)
            x = F.leaky_relu(x)

        x = self.hidden_layers[-1](x)
        minn, ila = x[:,:64], x[:,64:]

        minn = F.softmax(minn, dim=1)
        ila = F.softmax(ila, dim=1)

        return torch.cat([minn, ila], dim=1)

In [9]:
class discriminator_1(nn.Module):

    def __init__(self, conv_depth, hidden_size, hidden_depth):

        super().__init__()
        self.conv1 = nn.Conv2d(14, 64, 3, 1, padding=1, padding_mode = 'zeros')

        self.conv_layers = nn.ModuleList()
        self.hidden_layers = nn.ModuleList()

        self.conv_depth = conv_depth
        self.hidden_depth = hidden_depth
        self.hidden_size = hidden_size

        for _ in range(self.conv_depth):
            self.conv_layers.append(nn.Conv2d(64,64, 3, 1, padding=1, padding_mode = 'zeros'))
            self.conv_layers.append(nn.BatchNorm2d(64))
            self.conv_layers.append(nn.Conv2d(64,64, 3, 1, padding=1, padding_mode = 'zeros'))
            self.conv_layers.append(nn.BatchNorm2d(64))

        self.hidden_layers.append(nn.Linear(4096, hidden_size))
        
        self.hidden_layers.append(nn.Linear(hidden_size + 128, hidden_size))

        for _ in range(self.hidden_depth - 3):

            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))

        self.hidden_layers.append(nn.Linear(hidden_size, 1))
    

    def forward(self, board, move):

        x = self.conv1(board)
        
        for i in range(self.conv_depth):
            j = i*4
            ph = x.clone()
            ph = self.conv_layers[j](ph)
            ph = self.conv_layers[j+1](ph)
            ph = F.relu(ph)
            ph = self.conv_layers[j+2](ph)
            ph = self.conv_layers[j+3](ph)
            
            x = x + ph
            x = F.relu(x)
                  
                  
        x = torch.flatten(x, start_dim=1)

        for i in range(self.hidden_depth - 1):

            if i == 1:
                x = torch.cat((x, move), dim=1)
                
            x = self.hidden_layers[i](x)
            x = F.leaky_relu(x)

        x = self.hidden_layers[-1](x)
        x = F.sigmoid(x)
        
        return x

In [126]:
class GAN_1(nn.Module):
    #AI: 0, Human: 1
    def __init__(self, g_conv_depth, g_hidden_size, g_hidden_depth, d_conv_depth, d_hidden_size, d_hidden_depth, lr):
        
        super().__init__()
        
        print(device)
        
        self.generator = generator_1(g_conv_depth, g_hidden_size, g_hidden_depth)
        self.discriminator = discriminator_1(d_conv_depth, d_hidden_size, d_hidden_depth)
        
        self.logs = {"g_acc": [], "d_acc_r": [], "d_acc_f": [], 
                     "g_loss": [], "d_loss": [], 
                     "cur_g_loss": 0, "cur_d_loss": 0}
        
        self.made_loader = False
        
        self.configure_optimizers(lr)
        
    def forward(self, x):
        return self.generator(x)
    
    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)
    
    def train_step(self, train_boards, real_moves, train_generator):
        
        #train generator
        if train_generator:

            self.opt_g.zero_grad()
            
            fake_moves = self(train_boards)
                
            y_hat = self.discriminator(train_boards, fake_moves)
            
            y = torch.ones(real_moves.size(0), 1).to(device)
            
            g_loss = self.adversarial_loss(y_hat, y)
            self.logs["cur_g_loss"] += g_loss.item()
            
            g_loss.backward()

            self.opt_g.step()
              
            
        else:

            self.opt_d.zero_grad()
            
            y_hat_real = self.discriminator(train_boards, real_moves)
            y_real = torch.ones(real_moves.size(0), 1).to(device)
            
            d_real_loss = self.adversarial_loss(y_hat_real, y_real)
            
            y_hat_fake = self.discriminator(train_boards, self(train_boards).detach())
            y_fake = torch.zeros(real_moves.size(0), 1).to(device)
            
            d_fake_loss = self.adversarial_loss(y_hat_fake, y_fake)
            
            d_loss = d_real_loss + d_fake_loss
            self.logs["cur_d_loss"] += d_loss.item()
            
            d_loss.backward()

            self.opt_d.step()
            
    
    def configure_optimizers(self, lr):
        self.lr = lr
        self.opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, weight_decay=0.0001)
        self.opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr,weight_decay=0.0001)
    
    def on_epoch_end(self, epoch, G, val_data=None):
        
        self.logs["g_loss"].append(self.logs["cur_g_loss"] / G)
        self.logs["d_loss"].append(self.logs["cur_d_loss"] / G)
        
        print(f'Epoch {epoch+1} with g_loss: {self.logs["cur_g_loss"] / G} and d_loss: {self.logs["cur_d_loss"] / G}')
        
        self.logs["cur_g_loss"] = 0
        self.logs["cur_d_loss"] = 0
        
        if epoch % 1 == 0:
            
            if val_data is not None:
                
                val_boards = val_data.bitboards
                real_moves = val_data.moves
                
                fake_moves = self(val_boards)
                #fake_moves_ind = torch.argmax(fake_moves, dim=1)
                
                with torch.no_grad():
                    f_pred = self.discriminator(val_boards, fake_moves)
                    r_pred = self.discriminator(val_boards, real_moves)
                    
                    d_acc_f = torch.mean(torch.round(f_pred) == 0, dtype=torch.float).item()
                    d_acc_r = torch.mean(torch.round(r_pred) == 1, dtype=torch.float).item()
                    
                    d_dist_f = torch.mean(torch.abs(f_pred))
                    d_dist_r = torch.mean(torch.abs(1 - r_pred))
                    
                    g_acc = torch.mean((real_moves == torch.round(fake_moves)).all(dim=1), dtype=torch.float).item()
                
                print(f'Epoch: {epoch+1}, {g_acc=}, {d_acc_f=}, {d_acc_r=}')
                print(f"{d_dist_f=}, {d_dist_r=}")
                
                self.logs["d_acc_f"].append(d_acc_f)
                self.logs["d_acc_r"].append(d_acc_r)
                self.logs["d_dist_f"].append(d_dist_f)
                self.logs["d_dist_r"].append(d_dist_r)
                self.logs["g_acc"].append(g_acc)
                
        
            torch.save(self.generator, f"generator {epoch}.pt")
            torch.save(self.discriminator, f"discriminator {epoch}.pt")
            
    def create_dataloader(self, boards, meta, moves, B, N, N_val):

        if self.made_loader:

            clear_cuda()
            
        loader = DataLoader(GANData(boards[:N], meta[:N], moves[:N]), batch_size = B, shuffle = True, generator=torch.Generator(device=device))
        val_loader = GANData(boards[N:N+N_val], meta[N:N+N_val], moves[N:N+N_val])
        
        self.made_loader = True
        
        return loader, val_loader
        

        

In [11]:
class GANData(Dataset):

    def __init__(self, bitboards, white_turn, moves):

        self.bitboards = torch.tensor(bitboards, dtype = torch.float).to(device)

        self.moves = torch.zeros((self.bitboards.size(dim=0), 128), dtype = torch.float).to(device)

        for ind, move in tqdm(enumerate(moves), total=len(moves)):

            minn = move.from_square
            ila = move.to_square
        
            if not white_turn[ind]:
                minn = (63 - minn) // 8 * 8 + minn % 8
                ila = (63 - ila) // 8 * 8 + ila % 8

            self.moves[ind,minn] = 1
            self.moves[ind, ila + 64] = 1


    def __len__(self):

        return self.moves.size(dim=0)


    def __getitem__(self, idx):

        return self.bitboards[idx], self.moves[idx]
    

In [16]:
clear_cuda()

<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> torch.Size([64]) cuda:0
<class 'torch.Tensor'> to

In [93]:
del RSv1

In [94]:
RSv1 = GAN_1(g_conv_depth=6, g_hidden_size=1024, g_hidden_depth=2, 
             d_conv_depth=4, d_hidden_size=512, d_hidden_depth=3,
             lr=0.001).to(device)

cuda


In [17]:
loader, val_data = RSv1.create_dataloader(boards, meta, moves, B = 512, N=2_300_000, N_val=1_000)
G = len(loader)

100%|██████████| 1000/1000 [00:00<00:00, 13151.30it/s]


In [80]:
del RSv1.discriminator

RSv1.discriminator = discriminator_1(conv_depth=4, hidden_size=512, hidden_depth=3)
RSv1.configure_optimizers(0.001)

In [84]:
del RSv1.generator

RSv1.generator = generator_1(conv_depth=6, hidden_size=1024, hidden_depth=2)
RSv1.configure_optimizers(0.001)

In [75]:
generator_1?

In [110]:
train_discriminator = False
train_all = True

In [142]:
for epoch in range(51,150):

    reps = 0

    if train_all or train_discriminator:

        while RSv1.logs['d_acc_f'][-1] < 0.5:
            reps += 1
            if reps > 5:
                train_all = False
                train_discriminator = False
                break
            for bitboards, mvs in tqdm(loader):

                RSv1.train_step(bitboards, mvs, train_generator=False)


            RSv1.on_epoch_end(epoch, G, val_data)

    reps = 0
    if train_all or not train_discriminator:

        while RSv1.logs['d_acc_f'][-1] > 0.5:
            reps += 1
            if reps > 12:
                train_all = False
                train_discriminator = True
                break

            i=0
            for bitboards, mvs in tqdm(loader):

                if i > G // 8:
                    break

                RSv1.train_step(bitboards, mvs, train_generator=True)
                i += 1

            RSv1.on_epoch_end(epoch, G, val_data)

100%|██████████| 4493/4493 [03:03<00:00, 24.45it/s]


Epoch 52 with g_loss: 0.0 and d_loss: 0.028107999774310815
Epoch: 52, g_acc=0.003000000026077032, d_acc_f=1.0, d_acc_r=0.9970000386238098


 13%|█▎        | 562/4493 [00:23<02:41, 24.37it/s]


Epoch 52 with g_loss: 0.002368966074145966 and d_loss: 0.0
Epoch: 52, g_acc=0.0, d_acc_f=0.0, d_acc_r=0.9970000386238098


100%|██████████| 4493/4493 [03:02<00:00, 24.59it/s]


...

Epoch 90 with g_loss: 0.0 and d_loss: 0.03257294662090059
Epoch: 90, g_acc=0.004000000189989805, d_acc_f=1.0, d_acc_r=0.9960000514984131


 13%|█▎        | 562/4493 [00:23<02:46, 23.60it/s]


Epoch 90 with g_loss: 0.0036342408468751364 and d_loss: 0.0
Epoch: 90, g_acc=0.0, d_acc_f=0.0, d_acc_r=0.9960000514984131


  2%|▏         | 87/4493 [00:03<03:14, 22.61it/s]


KeyboardInterrupt: 

In [150]:
old_generator = torch.load("generator 85.pt")
torch.mean(RSv1.discriminator(val_boards, old_generator(val_boards)))

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 14.42 GiB is allocated by PyTorch, and 67.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [140]:
for epoch in range(50):
    
    reps = 0
    
    if train_all or train_discriminator:
        
        while RSv1.logs['d_acc_f'][-1] < 0.5:
            reps += 1
            if reps > 5:
                train_all = False
                train_discriminator = False
                break
            for bitboards, mvs in tqdm(loader):
        
                RSv1.train_step(bitboards, mvs, train_generator=False)
            

            RSv1.on_epoch_end(epoch, G, val_data)

    reps = 0
    if train_all or not train_discriminator:
        
        while RSv1.logs['d_acc_f'][-1] > 0.5:
            reps += 1
            if reps > 12:
                train_all = False
                train_discriminator = True
                break
                
            i=0
            for bitboards, mvs in tqdm(loader):
                
                if i > G // 8:
                    break
                
                RSv1.train_step(bitboards, mvs, train_generator=True)
                i += 1
            
            RSv1.on_epoch_end(epoch, G, val_data)

100%|██████████| 4493/4493 [03:02<00:00, 24.58it/s]


Epoch 1 with g_loss: 0.0 and d_loss: 0.024132794972129612
Epoch: 1, g_acc=0.0020000000949949026, d_acc_f=1.0, d_acc_r=1.0


 13%|█▎        | 562/4493 [00:23<02:41, 24.40it/s]


Epoch 1 with g_loss: 0.0024971719830587076 and d_loss: 0.0
Epoch: 1, g_acc=0.0, d_acc_f=0.0, d_acc_r=1.0


100%|██████████| 4493/4493 [03:03<00:00, 24.51it/s]


...

Epoch 50 with g_loss: 0.0 and d_loss: 0.010246130882807146
Epoch: 50, g_acc=0.0020000000949949026, d_acc_f=1.0, d_acc_r=0.9980000257492065


 13%|█▎        | 562/4493 [00:23<02:42, 24.22it/s]


Epoch 50 with g_loss: 1.012568757286216 and d_loss: 0.0
Epoch: 50, g_acc=0.0020000000949949026, d_acc_f=1.0, d_acc_r=0.9980000257492065


 13%|█▎        | 562/4493 [00:24<02:50, 23.07it/s]

Epoch 50 with g_loss: 0.39470355253507655 and d_loss: 0.0
Epoch: 50, g_acc=0.003000000026077032, d_acc_f=0.0, d_acc_r=0.9980000257492065


In [135]:
val_boards = val_data.bitboards
real_moves = val_data.moves

minn, ila = RDv2(val_boards)
fake_moves = torch.cat([F.softmax(minn, dim=1), F.softmax(ila,dim=1)], dim=1)
torch.mean((real_moves == torch.round(fake_moves)).all(dim=1), dtype=torch.float).item()

0.2510000169277191

In [137]:
fake_moves = RSv1(val_boards)
print(fake_moves[0])

#fake_moves_ind = torch.argmax(fake_moves, dim=1)
print(torch.round(fake_moves[0]))

g_acc = torch.mean((real_moves == torch.round(fake_moves)).all(dim=1), dtype=torch.float).item()

tensor([1.7404e-42, 0.0000e+00, 2.2697e-41, 2.9514e-19, 2.4398e-16, 4.8634e-12,
        9.2207e-27, 3.0531e-33, 0.0000e+00, 0.0000e+00, 2.2381e-24, 2.3693e-12,
        1.0543e-04, 2.4633e-19, 4.6248e-19, 9.9989e-01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0044e-09, 2.2264e-06, 1.8982e-10, 8.1555e-16, 6.4270e-11,
        1.2278e-31, 0.0000e+00, 1.0315e-31, 1.0168e-20, 5.1734e-15, 8.6261e-07,
        3.3822e-08, 5.3728e-09, 0.0000e+00, 0.0000e+00, 8.4442e-37, 1.1300e-11,
        1.6480e-18, 1.1017e-12, 9.6544e-10, 5.9234e-10, 0.0000e+00, 0.0000e+00,
        3.2209e-39, 1.5248e-36, 2.6022e-42, 5.4401e-41, 0.0000e+00, 4.2039e-45,
        0.0000e+00, 0.0000e+00, 7.8557e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+

In [141]:
test_against(lambda x: network_agent_prob_conv(x, RDv2), lambda x: network_agent_prob_conv(x, RSv1.generator), N=100)

100%|██████████| 50/50 [00:27<00:00,  1.80it/s]


(100, 0, 0, 1.0)

In [15]:
torch.cuda.memory_allocated() 

7274496